https://cscherrer.github.io/post/max-profit/

In [ ]:
import pymc3 as pm

import numpy as np

import pylab as plt

In [ ]:
# Price default when demand default

p0 = [10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0, 100.0]

q0 = [34, 32, 478, 973, 541, 26, 205, 772, 249, 3]


In [ ]:
with pm.Model() as m:
    
    loga = pm.Cauchy('loga',0, 10)
    
    c = pm.Cauchy('c',0, 10)
    
    logμ0 = loga + c * np.log(p0)
    
    μ0 = pm.Deterministic('μ0', np.exp(logμ0))
    
    qval = pm.Poisson('q', μ0, observed=q0)
    
    

In [ ]:
loga

In [ ]:
c

In [ ]:
logμ0

In [ ]:
μ0

In [ ]:
qval

In [ ]:
with m:
    trace = pm.sample()

In [ ]:
pm.summary(trace)

In [ ]:
with pm.Model() as m2:
    α = pm.Cauchy('α',0,10)
    β = pm.Cauchy('β',0,10)
    logμ0 = α + β * (np.log(p0) - np.log(p0).mean())
    μ0 = pm.Deterministic('μ0',np.exp(logμ0))
    qval = pm.Poisson('q0',μ0,observed=q0)

In [ ]:
with m2:
    t = pm.sample()
pm.summary(t)

In [ ]:
pm.traceplot(t);


In [ ]:
pm.forestplot(t)


In [ ]:
p = np.linspace(25,55)
μ = np.exp(t.α + t.β * (np.log(p).reshape(-1,1) - np.log(p0).mean()))
plt.plot(p,μ,c='k',alpha=0.01);
plt.plot(p0,q0,'o',c='C1');

In [ ]:
import scipy

plt.boxplot(scipy.stats.poisson.cdf(q0,mu=t['μ0']));


In [ ]:
np.mean(q0  > np.random.poisson(t['μ0']), 0)

In [ ]:
k = 20
π = (p - k).reshape(-1,1) * μ

plt.plot(p,π,c='k',alpha=0.01);
plt.plot(p,np.mean(π,1).T,c='C1',lw=2,label="$\mathbb{E}[\pi|P]$");
plt.fill_between(p,(np.mean(π,1)-np.std(π,1)).T,(np.mean(π,1)+np.std(π,1)).T,alpha=0.1,color='C1')
plt.plot(p,(np.mean(π,1)+np.std(π,1)).T,c='C1',lw=1,label="$\mathbb{E}[\pi|P]\ \pm$1 sd");
plt.plot(p,(np.mean(π,1)-np.std(π,1)).T,c='C1',lw=1);
pmax = p[np.argmax(np.mean(π,1))]
plt.vlines(pmax,300,900,colors='C0',linestyles='dashed',label="argmax$_P\ \mathbb{E}[\pi|P]$")

plt.ylim(300,900);
plt.xlabel("Price $P$")
plt.ylabel("Profit $\pi$")

plt.legend();